## Advanced SQL techniques for data scientists

This document contains notes from the course advanced SQL techniques for data scientists found on LinkedIn Learning from the following link.
- https://www.linkedin.com/learning/advanced-sql-for-data-scientists-13972889/advanced-sql-techniques-for-data-science?u=2046164

In [12]:
# install libraries for using SQL in jupyter notebooks
!pip install sql
import sql
print('importing completed')

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
importing completed


### data modeling: tables

#### Normalization
- minimize the risk of data anomolies
data anomolies: error or incosistencies in the data
update anomaly
insertion anomaly
deletion anomaly

avoiding anomalies by using normalization rules
- First normal form: values in each column of a table must be atomic.
- second normal form: all attributes not part of the key depend on the key.
- third normal form: no transitive dependencies.


OLTP | analytical |
--- | --- |
online transaction processing | data analysis | 
many reads and writes | *many reads by many processes |
many processes | many writes by few processes |
often normalized to 3rd form | often denormalized |

#### Denormalization
- improve read performance by reducing normalization

Note that this applies mostly for analytical databases. In exchange of not being normalized we get much better performance. Analytical databases have relatively low risk of anomalies because there are few updates, only have batch inserts, streaming inserts: simple data structures, no need for complex joins. 

Data is often redundant.
non-atomic values.
Transitive dependencies.

denormalized: star schema
row level orientation is commen for the star schema.

denormalized: wide column

##### advantages of denormalization
- easy to query
- efficient queries
- simplified load procedures

#### Partitioning
Horizontal partitioning
- limit scans to subset of partitions
- local indexes for each partition
- efficient adding and deleting

vertical partitioning
- increase number of rows in data block
- global indexes for each partition
- can reduce I/O (input/ output)

Hash partitioning
- partition on modules of hash partition key
- data does not logically group into sub-groups

###### example partitioning

CREATE TABLE test_table PARTITION OF table.origin
    FOR VALUES FROM ('2021-01-01') TO ('2021-05-01');

#### Materialized views
- persisted results of a query
- execute a query once: save results once -> read many times
- a form of caching
- trading space for time

When to use:
- long running queries
- complex queries, especially joins
- compute aggregates or other derived data
- seperate read and write operations
When not to use:
- eventual consistency
- cost of the update process
- concurrent reads during update? 
- size of marerialized view data
- refresh frequency

##### example materialized view
CREATE MATERIALZIED VIEW DB.table as
SELECT ...
FROM ...
(any SQL code can be added)

#### Read Replicas
- When data is written to the primary DB it is also stored in the read replica. 
QUeries can de directed to the read replica.
This way the Primary DB can focus on the write computations while the replica helps with ofloading read computations. 


# Data Modeling: indexes

Indexing for analytics queries
- reduces need to scan data blocks
- comes at cost of additional wirtes during data laoding
- the higher the cardinality of indext data, the better the performance imrprovement
- Not used in analytical data bases, suc as gogole redshift


#### B-tree indexes
B: balanced
- capture small amounts of data
- work well in many different cases
- ability to look up values in log time

#### Bitmap indexes
- used when there is a small number of possible values ij a column
- filter by bitwise operations such as AND, OR and NOT
- Time to access is based on time to perform bitwise operations
- read intensive use cases, few writes

Bitmap index availability
- some database solutions enable to build bitmap indexes explicitly
- postgreSQL not, it does it when possible

#### Hash index
- size of hash value depends on algorithm used
- no order preserving with hash functions
- similar inputs have vastly different outputs

basically each hash value will be different.
Use hash indexes:
- equality operations only
- can be smaller than b-tree
- comparable with speed of b-tree

#### GIST and SP-GIST indexes
These are only available in postgreSQL. Used as template to implement other indexes. Only used for special data types, discussed later on.
generalized search tree
used in PostgreSQL for indexing
SP gist: space partitioned GiST

#### GIN and BRIN indexes
These are only available in postgreSQL.
GIN: Generalized Inverted Index
- used when data to be indexed are composite values
- composite values require index to search for elements within composite item

insertions can be slow.
BRIN: block range index
- for very large tables
- column data has correlation with physical location (postal code)
- BRIN indexes store summary information about block ranges
- BRIN indexes tend to be small


# Query Optimization

GENERATE statement
select * from generate_series(1,100) as tl
- generate a list of 100 integers.
- instead of a general list we can specify specific timeseries (with time increments we can specify).
- this can be used to generate data for tables.

Use EXPLAIN before a query to get info about the QUERY plan.
- this is useful to see what the query actually does and in which order or parralel.
- EXPLAIN is also useful to review how good joins are.


Ways to join
- hash join (if you work with a lot of data)
- nested loops (good for small data set
- sort merge (if you have a lot of data and the output needs to be sorted)

Run ANALYZE to ensure statistics are up to date.

# User-Defined Functions